In [1]:
import os


from dotenv import load_dotenv
load_dotenv()

#from langchain_openai import AzureChatOpenAI
from langchain.chat_models import ChatOpenAI

/Users/deepakmishra/work/env/dev/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from datetime import datetime, timedelta
from typing import List
import math
import faiss
import logging
logging.basicConfig(level=logging.ERROR)
from langchain.docstore import InMemoryDocstore
#from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.embeddings import OpenAIEmbeddings

In [3]:
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.vectorstores import FAISS

from langchain_experimental.generative_agents import GenerativeAgent, GenerativeAgentMemory


In [4]:
USER_NAME = "Deepak"

In [8]:
# llm = AzureChatOpenAI(deployment_name=os.getenv("AZURE_DEP_NAME"),
#                          openai_api_version=os.getenv("AZURE_VERSION"),
#                           openai_api_key=os.getenv("AZURE_KEY"),
#                            azure_endpoint=os.getenv("AZURE_ENDPOINT"),
#                      max_tokens=1500)

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
#print(os.getenv("OPENAI_API_KEY"))
llm = ChatOpenAI(temperature=.7, model_name="gpt-3.5-turbo", max_tokens=1500)


In [9]:
def relevance_score_fn(score:float) -> float:
    """Returns similarity score on scale [0,1]"""
    return 1.0 - score/ math.sqrt(2)

In [10]:
from dotenv import load_dotenv
load_dotenv()
# os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_KEY")
# os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_ENDPOINT")
# os.environ["api_key"] = os.getenv("AZURE_KEY")

#from langchain_openai import AzureOpenAIEmbeddings



def create_new_memory_retriever():
    """ Create new vector store retriever unique to the agent """
    # embeddings_model = AzureOpenAIEmbeddings(
    #                 azure_deployment=os.getenv("AZURE_DEP_NAME"),
    #                 openai_api_version=os.getenv("AZURE_VERSION"),
    #                 openai_api_key = os.getenv("AZURE_KEY"),
    #                 azure_endpoint = os.getenv("AZURE_ENDPOINT")
    # )
    embeddings_model = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))
    
    embedding_size = 1536
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {}, relevance_score_fn=relevance_score_fn)
    return TimeWeightedVectorStoreRetriever(vectorstore=vectorstore, other_score_keys=["importance"], k=15)


#create_new_memory_retriever()


In [13]:

from dotenv import load_dotenv
load_dotenv()

os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_ENDPOINT")

os.environ["api_key"] = os.getenv("AZURE_KEY")

#print(os.getenv("AZURE_KEY"))

# from langchain_openai import AzureOpenAIEmbeddings


# embeddings_model = AzureOpenAIEmbeddings(
#                     azure_deployment=os.getenv("AZURE_DEP_NAME"),
#                     openai_api_version=os.getenv("AZURE_VERSION"),
#                     openai_api_key = os.getenv("AZURE_KEY"),
#                     azure_endpoint = os.getenv("AZURE_ENDPOINT"),
#                     model = "gpt-35-turbo-16k"
# )





In [14]:
alexis_memory = GenerativeAgentMemory(llm=llm,
                                      memory_retriever=create_new_memory_retriever(),
                                      verbose=False,
                                      reflection_threshold=8)

In [15]:
alexis = GenerativeAgent(
    name="Alexis",
    age=30,
    traits = "curious, creative writer, world traveller",
    status= "exploring the intersection of technology and storytelling",
    memory_retriever = create_new_memory_retriever(),
    llm=llm,
    memory = alexis_memory,
)

In [16]:
print(alexis.get_summary())

Name: Alexis (age: 30)
Innate traits: curious, creative writer, world traveller
Alexis is a hardworking, organized, and detail-oriented individual with strong communication and problem-solving skills. She is dedicated to her work and always strives for excellence.


In [17]:
alexis_observations = [
    "Alexis recalls her morning walk in the park",
    "Alexis feels excited about the new book she started reading",
    "Alexis remembers her conversation with a close friend",
    "Alexis thinks about the painting she saw at the art gallery",
    "Alexis is planning to learn a new recipe for dinner",
    "Alexis is lookin forward to her weekend trip",
    "Alexis contemplates her goals for the month."
]

for observation in alexis_observations:
    alexis.memory.add_memory(observation)

print(alexis.get_summary(force_refresh=True))

Name: Alexis (age: 30)
Innate traits: curious, creative writer, world traveller
Alexis is reflective, goal-oriented, enjoys nature and art, looks forward to new experiences and learning opportunities.


In [18]:
def interview_agent(agent: GenerativeAgent, message: str) -> str:
    """ Help the notebook user interact with the agent """
    new_message = "{0} says {1}".format(USER_NAME, message)
    return agent.generate_dialogue_response(new_message)[1]
    

In [19]:
interview_agent(alexis, "What do you like to do?")

'Alexis said "I enjoy writing, travelling, exploring nature, and experiencing new things. How about you, Deepak?"'

In [20]:
alexis_observations_day = [
    "Alexis starts her day with a refreshing yoga session.",
    "Alexis spends her time writing in her journal.",
    "Alexis experiments with a new recipe she found online."
]

for observation in alexis_observations_day:
    alexis.memory.add_memory(observation)

In [22]:
from termcolor import colored
for i, obeservation in enumerate(alexis_observations_day):
    _, reaction = alexis.generate_reaction(obeservation)
    print(colored(obeservation, "green"), reaction)
    if ((i+1) % len(alexis_observations_day)) == 0:
        print("*" * 40)
        print(colored("After these observations, Alexis's Summary is :\n {0}".format(alexis.get_summary(force_refresh=True)), "blue"))
        print("*" * 40)

Alexis starts her day with a refreshing yoga session. Alexis feels energized and ready to tackle the day ahead.
Alexis spends her time writing in her journal. Alexis finds solace and inspiration in her journal writing.
Alexis experiments with a new recipe she found online. Alexis is excited to try out new flavors and expand her culinary skills.
****************************************
After these observations, Alexis's Summary is :
 Name: Alexis (age: 30)
Innate traits: curious, creative writer, world traveller
Alexis is introspective, goal-oriented, enjoys journal writing, yoga, trying new recipes, and exploring different experiences such as traveling, nature, and reading. She is open-minded and curious about the world around her.
****************************************


In [23]:
jordan = GenerativeAgent(
    name="Jordan",
    age=28,
    traits = "tech enthuist, avid gamer, foodie",
    status= "navigating the world of tech startups",
    memory_retriever = create_new_memory_retriever(),
    llm=llm,
    memory = alexis_memory,
)

In [24]:
jordan_observations = [
    "Jordan finished challenging coding project last night",
    "Jordan won local gaming tournament over the weekend",
    "Jordan tried new south indian restaurant and loved it"
    
]

for observation in jordan_observations:
    jordan.memory.add_memory(observation)

print(jordan.get_summary(force_refresh=True))

Name: Jordan (age: 28)
Innate traits: tech enthuist, avid gamer, foodie
Jordan is a multi-talented individual who excels in both gaming and coding. They are adventurous and open to trying new experiences, as seen by their love for trying new restaurants. Alexis, on the other hand, is creative and introspective, finding solace in journal writing, yoga, and exploring different forms of art and literature. They are also eager to learn, constantly seeking out new skills and knowledge through activities like cooking and reading.


In [25]:
def run_conversation(agents: List[GenerativeAgent], initial_observation: str) -> None:
    """ Runs conversation between agents """
    _, observation = agents[1].generate_reaction(initial_observation)
    print(observation)
    max_turns = 3
    turn = 0
    while turn<=max_turns:
        break_dialogue = False
        for agent in agents:
            stay_in_dialogue, observation = agent.generate_dialogue_response(observation)
            print(observation)
            # observation = "{0} said {1}".format(agent.name, reaction)
            if not stay_in_dialogue:
                break_dialogue = True
        if break_dialogue:
            break
        turn +=1

In [26]:
agents = [alexis, jordan]
ini_conversation = "Alexis said: Hey Jordan, I have been exploring how technology influence creativity lately. Since you are into tech, I was wondering if you have seen something that can help me to undersatand."
run_conversation(agents, ini_conversation)

Jordan can respond with "That's interesting, Alexis! Have you looked into VR art or AI-generated art?"
Alexis said "That's a great suggestion, Jordan! I haven't looked into VR art or AI-generated art yet, but I'll definitely check them out. Thanks for the tip!"
Jordan said "That's great to hear, Alexis! Let me know if you have any questions or need more recommendations. I'm always here to help with tech-related stuff. Enjoy exploring VR art and AI-generated art!"
Alexis said "Thank you, Jordan! I'll definitely reach out if I have any questions. I appreciate your support and guidance in exploring these new technologies. Have a great day!"
Jordan said "Thank you, Alexis! I'm glad I could help. Feel free to reach out anytime. Have a wonderful day!"


In [27]:
interview_agent(alexis, "What was your conversation with Jordan?")

'Alexis said "I was discussing my interest in exploring the intersection of technology and storytelling with Jordan. It\'s such a fascinating topic to dive into! How about you, Deepak? Have you explored any interesting intersections lately?"'

In [28]:
interview_agent(jordan, "What was your conversation with Alexis?")

'Jordan said "I was discussing the intersection of technology and storytelling with Alexis. It\'s such a fascinating topic to explore! How about you, Deepak? Have you been exploring any interesting intersections lately?"'

In [29]:
def run_competitive_trivia(agents: List[GenerativeAgent], questions: list[str]) -> None:
    for question in questions:
        print("Trivia Question :: {0}".format(question))
        for agent in agents:
            response = agent.generate_dialogue_response(question)[1]
            print("{0}'s Answer:: {1}".format(agent.name, response))


agents = [alexis, jordan]
questions = [
    "Who is father of AI?",
    "What is the capital of India?",
    "Where is Burj Khalifa?"
]
run_competitive_trivia(agents, questions)

Trivia Question :: Who is father of AI?
Alexis's Answer:: Alexis said "I'm not sure, who is the Father of AI?"
Jordan's Answer:: Jordan said "That's a great question! The Father of AI is often considered to be John McCarthy, who coined the term 'artificial intelligence' back in 1956. He was a pioneer in the field and made significant contributions to the development of AI. Have you heard of him before?"
Trivia Question :: What is the capital of India?
Alexis's Answer:: Alexis said "I believe the capital of India is New Delhi, known for its vibrant culture and historical landmarks."
Jordan's Answer:: Jordan said "That's correct, the capital of India is New Delhi! It's known for its rich culture and historical landmarks. Have you ever been there before?"
Trivia Question :: Where is Burj Khalifa?
Alexis's Answer:: Alexis said "It's located in Dubai, United Arab Emirates, and it's known for being the tallest building in the world. Have a great day!"
Jordan's Answer:: Jordan said "It's loca

In [30]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    )-> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = "{0}: ".format(self.name)
        self.reset()
    def reset(self) -> str:
        self.message_history = ["Here is the conversation so far"]

    def send(self) -> str:
        """
        Applied the chatmodel to the message history
        and return the message string
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content
    def receive(self, name: str, message: str) -> None:
        """
        Concatenate {message} spoken by {name} into message history
        """
        self.message_history.append("{0}: {1}".format(name, message))
        



In [34]:
from typing import Callable
class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function = Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function

    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} for {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step +=1


    def step(self) -> tuple[str, str]:
        # 1. Choose next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]
        # 2. next speaker sends message
        message = speaker.send()
        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)
        # 4. increment the time
        self._step +=1
        return speaker.name, message

In [36]:
from langchain_core.prompts import PromptTemplate

class BiddingDialogueAgent(DialogueAgent):
    def __init__(
        self,
        name,
        system_message: SystemMessage,
        bidding_template: PromptTemplate,
        model: ChatOpenAI,
    ) -> None:
        super().__init__(name, system_message, model)
        self.bidding_template = bidding_template

    def bid(self) -> str:
        """
        Ask chat model to output a bid to speak
        """
        prompt = PromptTemplate(
            input_variables = ["message_history", "recent_message"],
            template=self.bidding_template,
        ).format(
            message_history="\n".join(self.message_history),
            recent_message = self.message_history[-1],
        )
        bid_string = self.model([SystemMessage(content=prompt)]).content
        return bid_string

In [37]:
################## CHALLENGE


In [38]:
character_names = ["CTO", "CMO", "CEO", "Investor-Daniel", "Invester-Sandra"]
topic = "Startup pitch on startup focussed on energy drinks with no caffeine"
word_limit = 15

# define the simulation
game_description = f"""Here is the topic for the startup pitch to investors Sandra and Daniel: {topic}.
The participants are: {', '.join(character_names)}. """

In [40]:
import numpy as np
def select_next_speaker(step: int, agents=List[DialogueAgent]) -> int:
    bids = []
    for agent in agents:
        bid = ask_for_bid(agent)
        bids.append(bid)

    # ramdomly select among multiple agents with the same bid
    max_value = np.max(bids)
    max_indices = np.where(bids == max_value)[0]
    idx = np.random.choice(max_indices)
    print("Bids::")
    for i, (bid, agent) in enumerate(zip(bids, agents)):
        print("\t{0} bid: {1}".format(agent.name, bid))
        if i == idx:
            selected_name = agent.name
    print("Selected : {0}".format(selected_name))
    print("\n")
    return idx
    

In [41]:
characters = []
model = ChatOpenAI(temperature=0.4)
for character_name, character_system_message, bidding_template in zip(
    character_names, character_system_messages, character_bidding_templates
):
    characters.append(
        BiddingDialogAgent(
            name=character_name,
            system_message=character_system_messages,
            bidding_template=character_bidding_templates
        )
    )

NameError: name 'character_system_messages' is not defined

In [42]:
max_iters = 10
n = 0
simulator = DialogueSimulator(agents=characters, selection_function=select_next_speaker)
simulator.reset()

first_message = "CEO, CMO, CTO You can start pitching your ideas to out investor Sandra and Deniel"
simulator.inject("moderator", first_message)

print("Moderator: {0}".format(first_message))
print("\n")
while n < max_iters:
    name, message = simulator.step()
    print("{0}: {1}".format(name, message))
    print("\n")
    n +=1

Moderator: CEO, CMO, CTO You can start pitching your ideas to out investor Sandra and Deniel




ValueError: zero-size array to reduction operation maximum which has no identity

In [43]:
####### Agent Tools